In [3]:
from helper import *
from funcs import *

In [4]:
import pandas as pd 

train_data = pd.read_csv('gap-development.tsv', sep='\t')
test_data  = pd.read_csv('gap-test.tsv', sep='\t')

**tf.tsv**

always predict A-coref as True and B-coref as False

In [5]:
f_test = open("gap-test.tsv", "r")
result = open("tf.tsv", 'w')
test_data = f_test.readlines()

for line in test_data[1:]:
    attrs = line.split('\t')
    result.write("\t".join([attrs[0]] + ['True', 'False']) + "\n")

f_test.close()
result.close()

In [6]:
!python gap_scorer.py --gold_ts=gap-test.tsv --system_tsv=tf.tsv

gap_scorer.py:126: DeprecationWarning: 'U' mode is deprecated
  with open(filename, 'rU') as f:
Overall recall: 51.8 precision: 45.9 f1: 48.7
		tp 918	fp 1082
		fn 855	tn 1145
Masculine recall: 51.0 precision: 45.3 f1: 48.0
		tp 453	fp 547
		fn 436	tn 564
Feminine recall: 52.6 precision: 46.5 f1: 49.4
		tp 465	fp 535
		fn 419	tn 581
Bias (F/M): 1.03



**closer.tsv**

Predict closer coreference to pronoun as True

In [7]:
f_test = open("gap-test.tsv", "r")
result = open("closer.tsv", 'w')
test_data = f_test.readlines()

for line in test_data[1:]:
    attrs = line.split('\t')
    tf = ['True', 'False']
    if abs(int(attrs[3]) - int(attrs[5])) > abs(int(attrs[3]) - int(attrs[8])):
        tf = ['False', 'True']

    result.write("\t".join([attrs[0]] + tf) + "\n")

f_test.close()
result.close()

In [8]:
!python gap_scorer.py --gold_ts=gap-test.tsv --system_tsv=closer.tsv

gap_scorer.py:126: DeprecationWarning: 'U' mode is deprecated
  with open(filename, 'rU') as f:
Overall recall: 50.1 precision: 44.4 f1: 47.1
		tp 888	fp 1112
		fn 885	tn 1115
Masculine recall: 51.6 precision: 45.9 f1: 48.6
		tp 459	fp 541
		fn 430	tn 570
Feminine recall: 48.5 precision: 42.9 f1: 45.5
		tp 429	fp 571
		fn 455	tn 545
Bias (F/M): 0.94



In [9]:
print_sents(random=True)

development-853 Landrieu was reelected largely on the basis of her 79,000-vote plurality in Orleans Parish. She polled roughly 42,000 votes ahead of Terrell statewide, defeating her 52% to 48%. In a debate with Landrieu in 2002, the senator lashed out at Terrell and told her the Senate race would be ``her last campaign'', but it was not.

development-448 He inadvertently confronted a crowd of people gathered around a preacher, Nazir Ahmad Ali, who was to later become a pioneering missionary in Sierra Leone. Whilst Ali was preaching, a chief requested Salih to judge the message brought by Ali. With the exception of two things, which ``held his throat'', Salih found that the message coincided with the teachings of the Quran.

development-1055 Five rooms from her Park Avenue apartment were installed in a memorial building after her death in 1960, bringing Webb's collection of works by Monet, Manet, and Degas to the museum grounds. A large pastel by Mary Cassatt, showing a young Electra Ha

In [10]:
import pandas as pd

In [11]:
train_data = pd.read_csv('gap-development.tsv', sep='\t')
test_data = pd.read_csv('gap-test.tsv', sep='\t')
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ID              2000 non-null   object
 1   Text            2000 non-null   object
 2   Pronoun         2000 non-null   object
 3   Pronoun-offset  2000 non-null   int64 
 4   A               2000 non-null   object
 5   A-offset        2000 non-null   int64 
 6   A-coref         2000 non-null   bool  
 7   B               2000 non-null   object
 8   B-offset        2000 non-null   int64 
 9   B-coref         2000 non-null   bool  
 10  URL             2000 non-null   object
dtypes: bool(2), int64(3), object(6)
memory usage: 144.7+ KB


**subject_pronoun.tsv**

Predict farther coreference pronoun as True when pronoun is subject pronoun (he, she).
Predict closer coreference pronoun as True when pronoun is not subject pronoun, which is object or possessive pronoun(her, his, him).

In [12]:
pronouns = set(train_data['Pronoun'])
pronouns

{'He', 'Her', 'His', 'She', 'he', 'her', 'him', 'his', 'she'}

In [13]:
result = open("subject_pronoun.tsv", 'w')

for attrs in test_data.iloc:
    pro, pro_id = attrs[2:4]
    A, A_id = attrs[4:6]
    B, B_id = attrs[7:9]

    if pro.lower() in ['he', 'she']:
        if abs(pro_id - A_id) > abs(pro_id - B_id):
            tf = ['TRUE', 'FALSE']
        else:
            tf = ['FALSE', 'TRUE']
    else:
        if abs(pro_id - A_id) > abs(pro_id - B_id):
            tf = ['FALSE', 'TRUE']
        else:
            tf = ['TRUE', 'FALSE']

    result.write("\t".join([attrs[0]] + tf) + "\n")

f_test.close()
result.close()

In [14]:
!python gap_scorer.py --gold_ts=gap-test.tsv --system_tsv=subject_pronoun.tsv

gap_scorer.py:126: DeprecationWarning: 'U' mode is deprecated
  with open(filename, 'rU') as f:
Overall recall: 58.3 precision: 51.6 f1: 54.8
		tp 1033	fp 967
		fn 740	tn 1260
Masculine recall: 59.4 precision: 52.8 f1: 55.9
		tp 528	fp 472
		fn 361	tn 639
Feminine recall: 57.1 precision: 50.5 f1: 53.6
		tp 505	fp 495
		fn 379	tn 621
Bias (F/M): 0.96



In [15]:
false_case = 0
for attrs in test_data.iloc:
    if attrs[6] == attrs[9]:
        false_case += 1

false_case/len(test_data)

0.1135

# subject.tsv

draw a relation between two proper nouns with sentence structure

(e.g. Which one is the subject?)

In [16]:
import nltk
sent = "In the next pair, Mikhaylov equalled Grishin's time and they shared Olympic gold."
tagged_sent = nltk.pos_tag(nltk.word_tokenize(sent))

Classify train_data into several categories

1. When words are inside parenthesis
2. When words appear after the pronoun
 * When the pronoun is subject pronoun
 * When the pronoun is object/possessive pronoun
3. When the both coreferences are false
4. When only one word is subject
5. When the both words are subject
6. When none of the words are subject
 * When a word appears multiple times
 * When both word appears same time
    

In [17]:
import pandas as pd 

train_data = pd.read_csv('gap-development.tsv', sep='\t')
test_data  = pd.read_csv('gap-test.tsv', sep='\t')

## 1. When words are inside parenthesis
Assume when word is inside parenthesis, coreference is always false

In [18]:
def isparenthesis(row):
    sent = row['Text']
    A = row['A']
    A_id = row['A-offset']
    B = row['B']
    B_id = row['B-offset']

    return (not (sent[A_id-1] == "(" and sent[A_id+len(A)] == ")"),
            not (sent[B_id-1] == "(" and sent[B_id+len(B)] == ")"))

def only_parenthesis(row):
    return sum(isparenthesis(row)) == 1

In [19]:
parenthesis_data = train_data[train_data.apply(only_parenthesis, axis=1)]

In [20]:
print_sents(datas={'manual':True, 'data':parenthesis_data}, num_lines = 10)

development-562 He was named to the NCAA All-Tournament team in 1980, and two first All Star teams in 1981. He began his playing career with the Pittsburgh Penguins who drafted him in the 6th round of the 1979 NHL Entry Draft. Chorney was traded to the Los Angeles Kings for a 6th round choice (Stuart Marston) in the 1985 NHL Entry Draft. After spending a season with the Kings, he was signed by the Washington Capitals but failed to earn a spot on the roster.

development-373 Bea Binene as George (Campus Tomboy) - She plays the boyish character that caught up in the ``bestfriend syndrome'', she tries to hide her feelings for her best friend enemy and instead, helps him get closer to Chloe. Jake Vargas as Jepoy (Guitar Enthusiast) - the group's resident rocker. He is head-over-heals in love with Chloe (Lexi Fernandez) and tries to get closer to her through his best friend George.

development-1765 During the restaurant's 1980s heyday at the top of the Zagat's survey, it was mentioned in W

result : 22 out of 31 sentences are predicted correct --> accuracy : 71%

## 2. When words appear after the pronoun
Assume coreference of words coming after the pronoun is false

In [21]:
after_pronoun = []

for row in train_data.iloc:
    if row['Pronoun-offset'] < row['A-offset'] or row['Pronoun-offset'] < row['B-offset']:
        after_pronoun.append(row)

after_pronoun = pd.DataFrame(after_pronoun, columns=train_data.columns)

In [22]:
after_subject_pronoun = after_pronoun[after_pronoun['Pronoun'].isin(['he', 'she', 'He', 'She'])]
after_other_pronoun = after_pronoun[after_pronoun['Pronoun'].isin(['his', 'her', 'him', 'His', 'Her', 'Him'])]

In [23]:
print_sents(datas={'manual':True, 'data':after_subject_pronoun}, num_lines = 10)

development-336 After the horrific plane crash of Wisteria Lane, the housewives think of their lives different ways. If Susan had stayed married to Karl and gotten fat; Bree imagines if Orson had died and Karl had taken her away; Angie imagines if Mona had survived and told the police everything she knows about the Bolen secret; Gabrielle thinks of Celia's future where she turns up to be an old woman; and Lynette imagines if Patrick had lived, but learns that Patrick died and Paige survived.

development-1338 Tina is later annoyed when she discovers Shirley is allowing Sylvie to sleep all day to avoid caring for her, and when Shirley goes to prison to help alleviate Mick's debts, Tina accuses her of doing so just to avoid Sylvie. Tina then struggles on her own, and eventually decides to place her in care. The day before she is due to be asseessed, Tina throws Sylvie a party.

development-323 Following the death of Tewodros II's much loved first wife, Empress Tewabech Ali (who was Tiruw

In [24]:
def after_pronoun_predict(data):
    A_pred = []
    B_pred = []
    for row in data.iloc:
        A_value = False if row['A-offset'] > row['Pronoun-offset'] else True
        B_value = False if row['B-offset'] > row['Pronoun-offset'] else True

        A_pred.append(A_value)
        B_pred.append(B_value)

    data['A-pred'] = A_pred
    data['B-pred'] = B_pred

In [25]:
after_pronoun_predict(after_subject_pronoun)
after_pronoun_predict(after_other_pronoun)

<ipython-input-24-ceb8dec2fe17>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['A-pred'] = A_pred
<ipython-input-24-ceb8dec2fe17>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['B-pred'] = B_pred


In [26]:
print(accuracy(after_subject_pronoun))
print(accuracy(after_other_pronoun))

0.411214953271028
0.2133676092544987


Result : It doesn't really matter whether a word comes after the pronoun or not

## 3. When the both word coreference is false

Let's find out what they have in common.

In [27]:
false_cases = 0
for row in train_data.iloc:
    if row['A-coref'] == row['B-coref']:
        false_cases += 1

print(false_cases/len(train_data))
print(false_cases)

0.1005
201


There are more than 10 percent of these kind of cases

In [28]:
falses_data = train_data[(train_data['A-coref'] == False) & (train_data['B-coref'] == False)]

In [29]:
print_sents(datas={'manual':True, 'data':falses_data}, num_lines = 10)

development-1747 On 21 October 1710, Ashburnham married Lady Mary Butler (died 1713), the second daughter of James Butler, 2nd Duke of Ormonde. After her death, he married, on 25 July 1714, Henrietta, Dowager Countess of Anglesey and 4th Baroness Strange; they had one child, Henrietta (c.1716--1732), later 5th Baroness Strange.

development-969 The second single, released on August 5, 2016, was Hollywoodn't. The music video was once again directed by Santiago Felipe and explores the dark side of Hollywood. In the video, Needles portrays Elizabeth Short a.k.a. 'The Black Dahlia', Sharon Tate and Jayne Mansfield. Her third studio album, Battle Axe, debuts on October 6, 2017, along with a video for the single of the same name.

development-1271 Otto I (c. 1128 -- July 8, 1184) was the second Margrave of Brandenburg, from 1170 until his death. Otto I was born into the House of Ascania as the eldest son of Albert I (``Albert the Bear''), who founded the Margraviate of Brandenburg in 1157, a

We can see that there are some patterns in both false sentences

1. There is another pronoun appeared before.
2. when both word is behind the pronoun, and the subject of the sentence is NNP.

1. There is another pronoun appeared before

In [30]:
def prior_pronoun(row):
    token_dict = tokenized_index(row)

    words = token_dict['words']
    word_index = token_dict['word_index']

    male = ['He', 'His', 'Him', 'he', 'his', 'him']
    female = ['She', 'Her', 'she', 'her', 'hers']

    pro_id = word_index[row['Pronoun-offset']]
    A_id = word_index[row['A-offset']]
    B_id = word_index[row['B-offset']]

    if row['Pronoun'] in male:
        for i, word in enumerate(words[:pro_id]):
            if word in male and i < min(A_id, B_id):
                return True
        return False
    elif row['Pronoun'] in female:
        for i, word in enumerate(words[:pro_id]):
            if word in female and i < min(A_id, B_id):
                return True
        return False

    else:
        print("Unexpected pronoun : {}", row['Pronoun'])

In [31]:
prior_pronoun_data = apply_model(train_data, [prior_pronoun], lambda x: (False, False))

1141 0.10429447852760736


In [32]:
def another_subject(row):
    token_dict = tokenized_index(row)

    tagged_sents = token_dict['tagged_sents']
    sents = token_dict['sents']
    word_index = token_dict['word_index']
    sent_index = token_dict['sent_index']

    pro_sent_id = sent_index[word_index[row['Pronoun-offset']]]

    def first_noun(sent):
        for word, tag in sent:
            if tag.startswith('NN') or tag.startswith('PR'):
                return (word, tag)

    word, tag = first_noun(tagged_sents[pro_sent_id])
    if tag.startswith('NNP') and not word in [row['A'].split()[0], row['B'].split()[0]]:
        return True

    return False

In [33]:
def both_latter(row):
    if row['A-offset'] > row['Pronoun-offset'] and row['B-offset'] > row['Pronoun-offset']:
        return True

    return False

mask = train_data.apply(both_latter, axis=1)
both_latter_data = train_data[mask]

In [34]:
another_subject_data = apply_model(train_data, [both_latter, another_subject], lambda x: (False, False))

48 0.5833333333333334


In [35]:
print_sents(datas={'manual':True, 'data':another_subject_data}, num_lines = 5)

development-336 After the horrific plane crash of Wisteria Lane, the housewives think of their lives different ways. If Susan had stayed married to Karl and gotten fat; Bree imagines if Orson had died and Karl had taken her away; Angie imagines if Mona had survived and told the police everything she knows about the Bolen secret; Gabrielle thinks of Celia's future where she turns up to be an old woman; and Lynette imagines if Patrick had lived, but learns that Patrick died and Paige survived.

development-46 The feature-length documentary traced the journey of the song, Hava Nagila, from the Ukraine to YouTube. Released theatrically and screened at 80 Jewish film festivals, Hava Nagila:The Movie was either opening or closing night at more than half of those festivals. Director Dyanna Taylor's film about her grandmother, the photographer Dorothea Lange, Grab a Hunk of Lightning, aired on PBS's ``American Masters'' in 2014 and was produced by Grossman.

development-1187 In 1997, he conten

## 4. When only one word is subject
Assume that subject word is always true, and other word is always false

In [36]:
subject_data = train_data[train_data['Pronoun'].isin(['he', 'she', 'He', 'She'])]
other_data = train_data[train_data['Pronoun'].isin(['his', 'him', 'her', 'His', 'Him', 'Her'])]
print(len(subject_data))
print(len(other_data))

801
1199


In [37]:
row = test_data.iloc[1716]
print(row['Text'], row['A'], row['B'])

The status of the show itself is unknown as it was originally expected to premiere in late 2008, but production has since been pushed back indefinitely. In 2011, Berkley published Ask-Elizabeth, a self-help book for teenage girls which drew from the workshops she conducted for the Ask-Elizabeth program. Berkley Elizabeth


In [38]:
overall_analysis(train_data, [only_subject], [word_subject, latter_word])

100%|██████████| 10/10 [01:16<00:00,  7.62s/it]


,Pronoun,Length,Accuracy0,Accuracy1
0,He,87,0.885057,0.241379
1,She,115,0.921739,0.208696
2,His,18,0.777778,0.388889
3,Her,19,0.736842,0.210526
4,he,139,0.791367,0.374101
5,she,147,0.768707,0.435374
6,his,306,0.767974,0.535948
7,him,58,0.431034,0.534483
8,her,270,0.648148,0.607407
9,hers,0,0.000000,0.000000


## 5. When both words are subject
Assume that latter word is True, while the other is False

In [39]:
both_subject_data = train_data.apply(both_subject, axis=1)
both_subject_data = train_data[both_subject_data]
len(both_subject_data)

202

In [40]:
print_sents(datas={'manual':True, 'data':both_subject_data}, num_lines = 10)

development-1515 On the 8th of May 1992, Compton was surprisingly appointed as manager of Torquay, shortly after their relegation back to the bottom flight of the Football League. The expected push for promotion did not materialise and Torquay found themselves in a relegation battle, which, only after Neil Warnock was brought in and Compton had reverted to his old youth coach role, they survived.

development-607 He returned to St. Louis in 1837. That year, his half-brother Tom Tobin, then 14 years old, left with Charles and his colleague Ceran St. Vrain to return to Taos. Tom worked as a trapper and scout at Bent's Fort and in Taos. Along with his brother, Tobin worked at Simeon Turley's store, mill, and distillery at Arroyo Hondo.

development-1269 Going into the season, manager Rogers Hornsby was optimistic that they would have a better club with more power overall, citing Landrith as one of the new players that could help the 1953 Reds be a success. Hornsby made it clear that veter

In [41]:
overall_analysis(train_data, [both_subject], [latter_word, former_word, closer_word, farther_word])

100%|██████████| 10/10 [00:28<00:00,  2.88s/it]


,Pronoun,Length,Accuracy0,Accuracy1,Accuracy2,Accuracy3
0,He,10,0.500000,0.400000,0.500000,0.400000
1,She,14,0.571429,0.428571,0.500000,0.500000
2,His,4,0.750000,0.250000,0.750000,0.250000
3,Her,0,0.000000,0.000000,0.000000,0.000000
4,he,18,0.833333,0.055556,0.833333,0.055556
5,she,25,0.640000,0.200000,0.640000,0.200000
6,his,42,0.690476,0.261905,0.690476,0.261905
7,him,22,0.181818,0.681818,0.181818,0.681818
8,her,67,0.597015,0.358209,0.611940,0.343284
9,hers,0,0.000000,0.000000,0.000000,0.000000


## 6. When both words are not subject


In [42]:
none_subject_data = train_data.apply(none_subject, axis=1)
none_subject_data = train_data[none_subject_data]
len(none_subject_data)

639

In [43]:
overall_analysis(train_data, [none_subject], [latter_word, former_word, closer_word, farther_word])

100%|██████████| 10/10 [00:30<00:00,  3.07s/it]


,Pronoun,Length,Accuracy0,Accuracy1,Accuracy2,Accuracy3
0,He,41,0.390244,0.463415,0.439024,0.414634
1,She,50,0.280000,0.420000,0.240000,0.460000
2,His,11,0.181818,0.454545,0.181818,0.454545
3,Her,19,0.368421,0.368421,0.368421,0.368421
4,he,78,0.423077,0.371795,0.423077,0.371795
5,she,77,0.337662,0.402597,0.324675,0.415584
6,his,148,0.452703,0.290541,0.506757,0.236486
7,him,18,0.555556,0.222222,0.555556,0.222222
8,her,197,0.502538,0.319797,0.507614,0.314721
9,hers,0,0.000000,0.000000,0.000000,0.000000


In [44]:
overall_analysis(train_data, [none_subject, both_latter, another_subject], [lambda x: (False, False)])

100%|██████████| 10/10 [00:31<00:00,  3.17s/it]


,Pronoun,Length,Accuracy0
0,He,0,0.000000
1,She,0,0.000000
2,His,0,0.000000
3,Her,0,0.000000
4,he,2,0.500000
5,she,2,1.000000
6,his,13,0.692308
7,him,0,0.000000
8,her,4,0.750000
9,hers,0,0.000000


In [60]:
print_sents(datas={'manual':True, 'data':none_subject_data[none_subject_data.apply(lambda row: row['Pronoun'] in ['He','She','His','Her'], axis=1)]}, num_lines = 10)

development-1488 Released in November 1987, the album was produced by Peter Cetera and would be her last for 17 years. Recorded mainly in Los Angeles and Malibu, California, I Stand Alone was produced by Peter Cetera, the former singer and bass guitar player with American rock band Chicago. He duetted with F*ltskog on the track ``I Wasn't the One (Who Said Goodbye)'', (No.93 Billboard Hot 100 and No.19 Adult Contemporary in the U.S.) The co-producer of the album was Bruce Gaitsch, with whom F*ltskog was in a relationship at the time.

development-839 Rino Kawashima (PC), Kei Mizusawa (Drama CD/PSP/anime) Machi is a female teacher who teaches Japanese at Yuihime Girls' Academy. She is the homeroom teacher of Shingo's class, 2-T; Airi, Angelina, Sana and Hayata are also in the same class. She is shy and is not good at talking with young men, but she actually likes boys' love.

development-225 John Nixon (May 10, 1815 - June 3, 1899), English mining engineer and colliery proprietor, was b

1. when two words are close to each other, referred words are more likely to be coreferred word.

 * {True word} (,?) and {Pronoun}


In [46]:
def CC_pronoun(row):
    token_dict = tokenized_index(row)

    tagged_words = token_dict['tagged_words']
    word_index = token_dict['word_index']

    pro_index = word_index[row['Pronoun-offset']]
    A_index = word_index[row['A-offset']]
    B_index = word_index[row['B-offset']]

    def coref(index):
        if index+5 >= len(tagged_words):
            return False
        return (tagged_words[index+1][1] == 'CC' and pro_index == index + 2) or \
               (tagged_words[index+2][1] == 'CC' and pro_index == index + 3)

    return coref(A_index), coref(B_index)

def exist_CC_pronoun(row):
    return sum(CC_pronoun(row)) == 1

In [73]:
print_sents(datas={'manual':True, 'data':apply_model(train_data, [exist_CC_pronoun], CC_pronoun)}, num_lines = 10)

23 0.8695652173913043
development-1622 She's a mum first and foremost and she's got her identity from being a mum. (...) She's a quiet person. (...) She's strong and she's way classier than I'll ever be!'' Gabby is also portrayed as being ``quiet''. In the same interview, Oruche noted the differences between Gabby and her character Liberty Baker in Footballers Wives.

development-1321 However, Archie Mitchell uses Suzy to plant DNA results in a Christmas cracker, showing that Sean Slater is not the father of Roxy's baby. Suzy leaves before Sean finds the results. Phil discovers that *10,000 is missing and blames Suzy so he and Shirley go to find her. In the Queen Victoria, Shirley trips Suzy and she is knocked out.

development-757 Immediately after Blackwood's penalty, wide receiver Charlie Brown's 26-yard reception advanced the Redskins to the Dolphins 16-yard line. After calling their final timeout with 14 seconds left, Washington made one last attempt to score a touchdown before tr

In [71]:
train_data.iloc[478]['Text']

"Sarah Wadsworth (17 Mar 1649 - ) m. Jonathan Ashley Thomas Wadsworth (1651 -- Sep 1687) m. Elizabeth Rebecca Wadsworth (1653 - ) Some of William Wadsworth's Notable Descendants The following persons are the descendants of William's sons Hon. John Wadsworth or his half brother Capt. Joseph Wadsworth of Charter Oak fame."

2. Located in the beggining of a sentence.

In [48]:
def begin_sentence(row):

    token_dict = tokenized_index(row)
    tagged_sents = token_dict['tagged_sents']

    Subjects = []

    def first_NP(sent):

        first_np = ""
        index=0
        while sent[index][1].startswith('NN') or sent[index][1].startswith('W') or sent[index][1] in ["'s", 'DT', 'IN', 'CD']:
            while sent[index][1].startswith('NNP'):
                first_np += sent[index][0] + " "
                if index < len(sent) - 1:
                    index += 1
                else:
                    break

            if len(first_np) > 0:
                Subjects.append(first_np)

            if index < len(sent) - 1:
                index += 1
            else:
                break

    def tagged_split(tagged_sent):
        splitted = []
        index = 0
        
        while(index < len(tagged_sent)):
            try:
                new_index = tagged_sent[index:].index((',',','))
                splitted.append(tagged_sent[index:index+new_index])
                index += new_index+1
            except:
                splitted.append(tagged_sent[index:])
                return splitted

    for sent in tagged_sents:
        for sub_sent in tagged_split(sent):
        
            first_np = first_NP(sub_sent)

    def exist(word):
        for first_word in Subjects:
            if word in first_word:
                return True
            elif first_word in word:
                return True
        return False

    return (exist(row['A']), exist(row['B']))

def begin_sentence_word(row):
    return sum(begin_sentence(row)) == 1

In [49]:
def most_occurance(row):

    token_dict = tokenized_index(row)
    tagged_words = token_dict['tagged_words']

    index=0
    NPs = []
    NP_dict = {}

    while index < len(tagged_words):
        np = ""
        while index < len(tagged_words) and tagged_words[index][1].startswith('NNP'):
            np += tagged_words[index][0] + " "
            index += 1
        
        if len(np) > 0:
            NPs.append(np[:-1])

        index += 1

    def occurance(word):
        occ = 0
        for w in NPs:
            if w in word:
                occ += 1

        return occ

    for word in set(NPs):
        NP_dict[word] = occurance(word)

    max_occ = sorted(NP_dict.items(), key = (lambda x: x[1]), reverse=True)[0]

    if max_occ[1] > 1:
        return [NP for NP in NP_dict if NP_dict[NP] == max_occ[1]]
    else:
        return None

def more_occurance(row):

    most_occs = most_occurance(row)

    if most_occs == None:
        return (True, True)

    def exist(word, words):
        for w in words:
            if word in w or w in word:
                return True
        return False

    return (exist(row['A'], most_occs), exist(row['B'], most_occs))

def only_most_occurance(row):
    return sum(more_occurance(row)) == 1

def none_most_occurance(row):
    return sum(more_occurance(row)) == 0

def both_most_occurance(row):
    return more_occurance(row) == (True, True)

In [50]:
overall_analysis(train_data, [none_subject, only_most_occurance], [more_occurance])

100%|██████████| 10/10 [00:45<00:00,  4.53s/it]


,Pronoun,Length,Accuracy0
0,He,20,0.700000
1,She,16,0.562500
2,His,4,0.750000
3,Her,9,0.555556
4,he,26,0.653846
5,she,34,0.441176
6,his,59,0.576271
7,him,9,0.555556
8,her,92,0.597826
9,hers,0,0.000000


In [51]:
none_occurance_data = none_subject_data[none_subject_data.apply(none_most_occurance, axis=1)]

In [52]:
both_occurance_data = none_subject_data[none_subject_data.apply(both_most_occurance, axis=1)]

In [53]:
print(len(none_occurance_data))
print(len(both_occurance_data))

200
170


In [54]:
print_sents(datas={'manual':True, 'data':both_occurance_data[both_occurance_data.apply(lambda row: row['Pronoun'] in ['He','She','His','Her'], axis=1)]}, num_lines = 10)

development-839 Rino Kawashima (PC), Kei Mizusawa (Drama CD/PSP/anime) Machi is a female teacher who teaches Japanese at Yuihime Girls' Academy. She is the homeroom teacher of Shingo's class, 2-T; Airi, Angelina, Sana and Hayata are also in the same class. She is shy and is not good at talking with young men, but she actually likes boys' love.

development-1068 She agrees, and as she's crossing the street to return home, a car barrels down the road towards her. Gideon stops the impending accident and allows Abbie to cross the street without incident. The next day, the Graingers visit Jack's grandfather, Caleb. He gives the children presents:

development-1849 Kenney-Silver was born Kerri Kenney in Westport, Connecticut, the daughter of Sharon (McGaughey) and Larry Kenney, an actor who is known for his classic voice-over work. She has a sister, Ashley, and a brother, Tanner.

development-728 And often (what) adds a tremendous amount of comedy is the choosing to look over there and see w

In [55]:
overall_analysis(train_data, [none_subject, both_most_occurance], [former_word, latter_word, closer_word, farther_word, lambda row: (False, False)])

100%|██████████| 10/10 [00:32<00:00,  3.26s/it]


,Pronoun,Length,Accuracy0,Accuracy1,Accuracy2,Accuracy3,Accuracy4
0,He,9,0.555556,0.444444,0.444444,0.555556,0.000000
1,She,21,0.428571,0.142857,0.142857,0.428571,0.428571
2,His,2,0.500000,0.000000,0.000000,0.500000,0.500000
3,Her,3,0.000000,1.000000,1.000000,0.000000,0.000000
4,he,23,0.478261,0.217391,0.260870,0.434783,0.304348
5,she,20,0.400000,0.400000,0.350000,0.450000,0.200000
6,his,39,0.128205,0.615385,0.564103,0.179487,0.256410
7,him,3,0.000000,0.666667,0.666667,0.000000,0.333333
8,her,50,0.280000,0.560000,0.560000,0.280000,0.160000
9,hers,0,0.000000,0.000000,0.000000,0.000000,0.000000


In [74]:
print_sents(datas={'manual':True, 'data':none_occurance_data}, num_lines = 10)

development-542 Noteworthy projects that JESS3 has undertaken with Thomas's leadership include the NASA website ``Buzzroom'' that tracks NASA-related tweets, videos and images, the C-SPAN Video Library, which was awarded a Peabody Award in March 2011, and the design of The Conversation Prism, an infographic poster produced in partnership with Brian Solis. In 2010, he was approached by Facebook to develop the Facebook Stories application.

development-1507 Frederick Magnus (13 January 1672 -- 24 February 1672) Frederica Augusta (21 June 1673 -- 24 July 1674) Christina Sophia (17 December 1674 -- 22 January 1676) Klaudia Magdalene Elisabeth (15 November 1675 -- 18 April 1676) Catherine (10 October 1677 -- 11 August 1746), in 1701 she married Count Johann Friedrich von Leiningen-Hartenburg Charles III William, Margrave of Baden-Durlach (17 January 1679 -- 12 May 1738), he married Magdalena Wilhelmine of W*rttemberg Johanna Elisabeth (3 October 1680 -- 2 July 1757), in 1697 she married Ebe

In [58]:
overall_analysis(train_data, [none_subject, none_most_occurance], [former_word, latter_word, closer_word, farther_word, lambda row: (False, False)])

100%|██████████| 10/10 [00:30<00:00,  3.07s/it]


,Pronoun,Length,Accuracy0,Accuracy1,Accuracy2,Accuracy3,Accuracy4
0,He,12,0.333333,0.250000,0.250000,0.333333,0.416667
1,She,13,0.461538,0.230769,0.153846,0.538462,0.307692
2,His,5,0.600000,0.000000,0.000000,0.600000,0.400000
3,Her,7,0.285714,0.142857,0.142857,0.285714,0.571429
4,he,29,0.379310,0.379310,0.379310,0.379310,0.241379
5,she,23,0.391304,0.304348,0.260870,0.434783,0.304348
6,his,50,0.260000,0.340000,0.420000,0.180000,0.400000
7,him,6,0.166667,0.500000,0.500000,0.166667,0.333333
8,her,55,0.418182,0.345455,0.345455,0.418182,0.236364
9,hers,0,0.000000,0.000000,0.000000,0.000000,0.000000


In [59]:
rest = apply_model(train_data, [none_subject, none_most_occurance], begin_sentence)

200 0.35


In [61]:
overall_analysis(train_data, [], [subject])

100%|██████████| 10/10 [03:32<00:00, 21.21s/it]


,Pronoun,Length,Accuracy0
0,He,138,0.782609
1,She,179,0.765363
2,His,33,0.757576
3,Her,38,0.578947
4,he,235,0.689362
5,she,249,0.654618
6,his,496,0.699597
7,him,98,0.571429
8,her,534,0.602996
9,hers,0,0.000000


In [62]:
overall_analysis(test_data, [], [subject])

100%|██████████| 10/10 [02:31<00:00, 15.12s/it]


,Pronoun,Length,Accuracy0
0,He,127,0.842520
1,She,159,0.691824
2,His,28,0.642857
3,Her,37,0.675676
4,he,221,0.674208
5,she,237,0.687764
6,his,528,0.662879
7,him,96,0.468750
8,her,566,0.597173
9,hers,1,0.000000


In [63]:
subject_data = apply_model(test_data, [], subject)

2000 0.6525


In [68]:
result = open("subject_word.tsv", 'w')

for row in subject_data.iloc:

    result.write("\t".join([row['ID'], str(row['A-pred']), str(row['B-pred'])]) + "\n")

result.close()

In [69]:
!python gap_scorer.py --gold_ts=gap-test.tsv --system_tsv=subject_word.tsv

gap_scorer.py:126: DeprecationWarning: 'U' mode is deprecated
  with open(filename, 'rU') as f:
Overall recall: 72.0 precision: 66.5 f1: 69.2
		tp 1277	fp 642
		fn 496	tn 1585
Masculine recall: 73.3 precision: 68.6 f1: 70.9
		tp 652	fp 299
		fn 237	tn 812
Feminine recall: 70.7 precision: 64.6 f1: 67.5
		tp 625	fp 343
		fn 259	tn 773
Bias (F/M): 0.95

